# Getting Started with BlazingSQL
*By Winston Robson*

In this notebook, we will cover how to read and query CSV files with cuDF and BlazingSQL.

#### BlazingSQL install check
The next cell checks that you have BlazingSQL installed, and offers to install it if not (making sure the notebook will run as expected).

In [1]:
import sys 
# point import path notebooks-contrib/utils
sys.path.append('../../../utils/')
from sql_check import bsql_start
# check that BlazingSQL is installed
bsql_start()

"You've got BlazingSQL set up perfectly! Let's get started with SQL in RAPIDS AI!"

## create BlazingContext
You can think of BlazingContext much like a Spark Context (i.e. information such as FileSystems registered & Tables created will be stored).

In [6]:
from blazingsql import BlazingContext

bc = BlazingContext()

BlazingContext ready


## Read CSV
First we need to download a CSV file. Then we use cuDF to read the CSV file, which gives us a GPU DataFrame (GDF). 

To learn more about the GDF and how it enables end to end workloads on rapids, check out *[The GPU DataFrame (GDF) and cuDF in RAPIDS AI](https://blog.blazingdb.com/blazingsql-part-1-the-gpu-dataframe-gdf-and-cudf-in-rapids-ai-96ec15102240)*.

In [7]:
import cudf

# cudf (gpu) dataframe from csv 
gdf = cudf.read_csv('../../../data/blazingsql/music.csv')

# let's see how it looks
gdf.head()

,ARTIST,RATING,YEAR,LOCATION,FESTIVAL_SET
0,Arcade Fire,10.0,2018.0,Las Vegas,1.0
1,Justice,10.0,2018.0,Las Vegas,1.0
2,Florence and The Machine,10.0,2018.0,Las Vegas,1.0
3,Odesza,10.0,2018.0,Indio,1.0
4,Bon Iver,10.0,2017.0,Indio,1.0


## Create a Table
Now we just need to create a table. 

In [8]:
# BlazingSQL table from cuDF DataFrame 
bc.create_table('music', gdf)

## Query a Table
That's it! Now when you can write a SQL query the data will get processed on the GPU with BlazingSQL, and the output will be a GPU DataFrame (GDF) inside RAPIDS!

In [9]:
# query 10 events with a rating of at least 7 (type(result_gdf)==cudf.core.dataframe.DataFrame)
result_gdf = bc.sql('SELECT * FROM music where RATING >= 7 LIMIT 10')

# display dataframe 
result_gdf

,ARTIST,RATING,YEAR,LOCATION,FESTIVAL_SET
0,Arcade Fire,10.0,2018.0,Las Vegas,1.0
1,Justice,10.0,2018.0,Las Vegas,1.0
2,Florence and The Machine,10.0,2018.0,Las Vegas,1.0
3,Odesza,10.0,2018.0,Indio,1.0
4,Bon Iver,10.0,2017.0,Indio,1.0
5,LA Philharmonic + Sigur Ros,10.0,2017.0,LA,0.0
6,Sigur Ros,10.0,2014.0,Malmo,0.0
7,Arcade Fire,10.0,2014.0,Indio,1.0
8,Escort,9.0,2018.0,San Francisco,0.0
9,Phoenix,9.0,2018.0,Berkeley,0.0


In [10]:
# query events in San Francisco, CA
gdf = bc.sql("SELECT * FROM music where LOCATION = 'San Francisco'")

# sample 7 rows by converting to pandas 
gdf.to_pandas().sample(7)

,ARTIST,RATING,YEAR,LOCATION,FESTIVAL_SET
46,Chromeo,4.0,2014.0,San Francisco,1.0
4,Griz,8.0,2016.0,San Francisco,1.0
26,George Ezra,6.0,2012.0,San Francisco,1.0
40,Above and Beyond,4.0,2017.0,San Francisco,1.0
34,Halsey,5.0,2017.0,San Francisco,1.0
36,Broods,5.0,2011.0,San Francisco,1.0
43,Odesza,4.0,2014.0,San Francisco,1.0


# You're Ready to Rock
And... thats it! You are now live with BlazingSQL.


Check out our [docs](https://docs.blazingdb.com) to get fancy or to learn more about how BlazingSQL works with the rest of [RAPIDS AI](https://rapids.ai/).